In [1]:
!pip install transformers datasets accelerate peft bitsandbytes
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable


[notice] A new release of pip is available: 25.0 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip



Looking in indexes: https://download.pytorch.org/whl/cu118


In [1]:
import torch
#
# Use GPU if possible
#
torch.cuda.empty_cache()
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

Using device: cuda


In [1]:
from huggingface_hub import login

login()

In [37]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

#
# Load Model from HuggingFace
#
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
model = model.to(device)
print("Model loaded")

Model loaded


In [38]:
import json
import pandas as pd
import re
import os
from datasets import Dataset
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset
#
# 
# Format Data for English to Esgish translation and vice versa
#
#
#

# Path to Excel File
train_excel_path = "translated_queries_TEST2.xlsx"
train_df = pd.read_excel(train_excel_path)

# Excel file should have two columns, "Esgish", and "English"
expected_columns = ["Esgish", "English"]
train_df.rename(columns=lambda x: x.strip(), inplace=True)

for col in expected_columns:
    assert col in train_df.columns, f"Missing expected column: {col}"
print("Training data loaded successfully.")
print(train_df.head())

train_data = []

for _, row in train_df.iterrows():
    # Format Data for English to Esgish
    train_data.append({
        "input": f"Convert English to Esgish: {row['English']}",
        "output": row['Esgish']
    })
    # Format Data for Esgish to English
    train_data.append({
        "input": f"Convert Esgish to English: {row['Esgish']}",
        "output": row["English"]
    })

# Splits dataset into training data and other data for evaluation and test
train_list = Dataset.from_list(train_data)
split = train_list.train_test_split(test_size=0.2)
train_dataset = split['train']
temp_dataset = split['test']

# Split temp into validation / test
split_temp = temp_dataset.train_test_split(test_size=0.5, seed=42)
eval_dataset = split_temp['train']
test_dataset = split_temp['test']
print(train_dataset[0])

def preprocess_function(examples):
    inputs = tokenizer(examples["input"], truncation=True, padding="max_length", max_length=128)
    labels = tokenizer(examples["output"], max_length=128, truncation=True, padding="max_length")    
    model_inputs = {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "labels": labels["input_ids"], 
    }
    return model_inputs

train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=["input", "output"])
eval_dataset = eval_dataset.map(preprocess_function, batched=True, remove_columns=["input", "output"])
test_dataset = test_dataset.map(preprocess_function, batched=True, remove_columns=["input", "output"])
print(train_dataset[0])

Training data loaded successfully.
                                           Esgish  \
0      [AbortifacientsDistrMaxRevRatio1Y] < '0.1'   
1     [AbortifacientsDistrMaxRevRatio1Y] > '0.01'   
2        [AbortifacientsDistrMaxRevRatio3Y] > '0'   
3  [AbortifacientsInvolvement] ANY 'Distribution'   
4    [AbortifacientsInvolvement] ANY 'Production'   

                                             English  
0  All companies or issuers where the maximum sha...  
1  All companies or issuers where the maximum sha...  
2  All companies that have a maximum share of rev...  
3  All companies or issuers that are involved in ...  
4  All companies or issuers that produce abortifa...  
{'input': "Convert Esgish to English: AND([FossilFuelOilPowerRevShareMin] > '0.1',[FossilFuelOilExtractRevShareMin] > '0.05',[FossilFuelGasExtractRevShareMin] > '0.05',[OilSandsProdMinRev] > '0.01',[HydraulicFracturingShareMin] > '0.01',[NuclearPowerRevShareMin] > '0.1',[NuclearPowerUraniumRevShareMin] > '0.1')", '

Map:   0%|          | 0/694 [00:00<?, ? examples/s]

Map:   0%|          | 0/87 [00:00<?, ? examples/s]

Map:   0%|          | 0/87 [00:00<?, ? examples/s]

{'input_ids': [1193, 3027, 1122, 122, 1273, 12, 1566, 10, 3430, 599, 6306, 371, 32, 7, 7, 173, 371, 76, 15, 40, 667, 173, 23553, 1649, 208, 24501, 12858, 908, 2490, 3, 31, 16029, 31, 6, 6306, 371, 32, 7, 7, 173, 371, 76, 15, 40, 667, 173, 5420, 6471, 17, 1649, 208, 24501, 12858, 908, 2490, 3, 31, 25079, 31, 6, 6306, 371, 32, 7, 7, 173, 371, 76, 15, 40, 517, 9, 7, 5420, 6471, 17, 1649, 208, 24501, 12858, 908, 2490, 3, 31, 25079, 31, 6, 6306, 667, 173, 134, 232, 7, 3174, 26, 12858, 1649, 208, 908, 2490, 3, 31, 11739, 536, 31, 6, 6306, 566, 63, 3515, 83, 447, 371, 3738, 17, 7920, 24501, 12858, 908, 2490, 3, 31, 11739, 536, 31, 6, 6306, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [45]:
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq, T5Tokenizer, EarlyStoppingCallback
from transformers import AdamW, get_scheduler

training_args = TrainingArguments(
    output_dir="./test",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4, #Can be increased if GPU has more VRAM, originally was 4
    per_device_eval_batch_size=4, #Can be increased if GPU has more VRAM, originally was 4
    gradient_accumulation_steps=4,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

# Define the EarlyStoppingCallback
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,
)

# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Define scheduler
lr_scheduler = get_scheduler(
    name="linear",  # can be "linear", "cosine", "cosine_with_restarts", "polynomial", "constant"
    optimizer=optimizer,
    num_warmup_steps=0,  # or a few hundred, depending on your dataset size
    num_training_steps=len(train_dataset) * training_args.num_train_epochs // training_args.per_device_train_batch_size
)

# Initialize the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    processing_class=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, padding=True),
    optimizers=(optimizer, lr_scheduler),
    callbacks=[early_stopping],
)

print("Training started...")

trainer.train()
print("training done")

C:\Users\Jason Chen\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training started...


Epoch,Training Loss,Validation Loss
1,0.592700,0.534204
2,0.524200,0.505907
3,0.481200,0.484658
4,0.394000,0.462294


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


training done


In [46]:
# Save model after training, name can be changed
# If you open a trained model and trained it again, has to be a different name than the trained model
model.save_pretrained("./new_2")
tokenizer.save_pretrained("./new_2")

('./new_2\\tokenizer_config.json',
 './new_2\\special_tokens_map.json',
 './new_2\\spiece.model',
 './new_2\\added_tokens.json',
 './new_2\\tokenizer.json')

In [47]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Loads saved model
model = AutoModelForSeq2SeqLM.from_pretrained("./new_2")
tokenizer = AutoTokenizer.from_pretrained("./new_2")

In [48]:
import pandas as pd

#
# Translate random queries from English to Esgish2
#

excel = pd.read_excel("translated_queries_TEST2.xlsx")
sample = excel[["Esgish", "English"]].dropna().sample(5).values

for esg, eng in sample:
    input_text = f"Convert English to Esgish: {eng}"
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
    output_ids = model.generate(
        **inputs,
        max_length=1024,
        num_beams=4,
        early_stopping=True,
        no_repeat_ngram_size=3
    )
    print("Query: ", eng)
    print("Correct: ", esg)
    print("Model: ", tokenizer.decode(output_ids[0], skip_special_tokens=True), "\n\n")

Query:  All companies that are involved in the production of fossil fuels and have a minimum percentage of revenue from fossil fuel business activities of 5% or more.
Correct:  AND([FossilFuelInvolvement] ANY 'Production',[FossilFuelRevShareMin] >= '0.05')
Model:  AND([FossilFuelInvolvement] ANY 'Production|RevShareMin] > '0.05') 


Query:  All companies or issuers where the Board Chair is an executive or insider.
Correct:  [ChairmanClassification] == 'Executive/insider'
Model:  AND([BoardPresident] IS 'Executive',[Insider] IS IN 'INNER') 


Query:  All companies with a GMO - Agriculture Production Maximum Percentage of Revenues greater than 10% in the latest financial year.
Correct:  [GMOAgricultureProdMaxRev] > '0.1'
Model:  AND([GMOAgricultureProdMaxRev] > '0.1') 


Query:  All companies or issuers that have a minimum percentage of recent-year revenues for their involvement in coal mining of at least 0% and a minimum percentage of recent-year revenues for their involvement in coal e

In [49]:
#
# Translate random queries from Esgish2 to English
#
excel = pd.read_excel("translated_queries_TEST1.xlsx")
sample = excel[["Esgish", "English"]].dropna().sample(5).values

for esg, eng in sample:
    input_text = "Convert Esglish to English: " + esg
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
    output_ids = model.generate(
        inputs["input_ids"],
        max_length=1024,
        num_beams=4,
        early_stopping=True
    )
    print("Query: ", esg)
    print("Correct: ", eng)
    print("Model: ", tokenizer.decode(output_ids[0], skip_special_tokens=True), "\n\n")

Query:  [AFLCIOBoycottList] True
Correct:  All companies or issuers listed on the AFL-CIO Boycott List.
Model:  Please provide a list of all companies or issuers that have a BOYcott List of True. 


Query:  [CivFAAssaultWeaponsInvolvement] ANY 'Production|Distribution|Services'
Correct:  All companies or issuers involved in the production, distribution, or provision of services related to assault weapons.
Model:  All companies or issuers that are involved in the production, distribution, or provision of services related to assault weapons. 


Query:  [AdjustedEnterpriseValue] > '0'
Correct:  All companies with an adjusted enterprise value greater than zero.
Model:  Please provide a list of all companies or issuers that have an adjusted Enterprise Value greater than 0. 


Query:  [CCAMoneyLaunderingControversy] True
Correct:  All companies or issuers that have been flagged for having a money laundering concern in their country.
Model:  Please provide a list of all companies or issuers t